In [1]:
import argparse
import os
import sys
import pandas as pd
import optuna
from optuna.integration import PyTorchLightningPruningCallback

import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from glob import glob 

from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.profilers import PyTorchProfiler

sys.path.append(os.pardir)
from src.config import *
from src.dataset import DataModule
from src.model import LightningNetwork

In [ ]:

    # base_model_name = 'efficientnet_b0'
    # model = LightningNetwork(
    #         base_model=get_base_model(base_model_name),
    #         dropout=0.2, 
    #         output_dims=[128, 64],
    #         learning_rate=LEARNING_RATE
    # )
    # data_module = DataModule(model_name=base_model_name)

    # trainer = pl.Trainer(
    #     logger=True,
    #     # accelerator=ACCELERATOR,
    #     # devices=DEVICES,
    #     max_epochs=NUM_EPOCHS,
    #     # precision=PRECISION,
    #     # max_steps=5,
    #     # fast_dev_run=True
    #     # callbacks=[EarlyStopping(monitor="val_loss")],
    # )
    # trainer.fit(model, data_module)



In [ ]:
for i in range(32, 512, 32)

In [51]:
import torch
from torch import nn
from typing import List, Optional
import torch.nn.functional as F
from torch import nn, optim
import pytorch_lightning as pl
import torchmetrics



class Network(nn.Module):
    def __init__(self, base_model, dropout: float, output_dims: List[int]) -> None:
        super().__init__()

        self.base_model = base_model
        input_dim: int = base_model.classifier[1].in_features

        layers: List[nn.Module] = []
        for output_dim in output_dims:
            layers.append(nn.Linear(input_dim, output_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_dim = output_dim
        layers.append(nn.Linear(input_dim, NUM_CLASSES))
        layers.append(nn.Softmax(dim=1))

        self.base_model.classifier = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.base_model(x)


class LightningNetwork(pl.LightningModule):
    def __init__(self, 
                 base_model,
                 dropout: float, 
                 output_dims: List[int],
                 learning_rate: float) -> None:
    
        super().__init__()
        self.model = Network(base_model=base_model, dropout=dropout, output_dims=output_dims)
        self.learning_rate = learning_rate
        self.loss_fn = nn.CrossEntropyLoss()

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=NUM_CLASSES)
        self.recall = torchmetrics.Recall(task="multiclass", num_classes=NUM_CLASSES)
        self.precision = torchmetrics.Precision(task="multiclass", num_classes=NUM_CLASSES)
        self.f1_score = torchmetrics.F1Score(task="multiclass", num_classes=NUM_CLASSES)
        self.auc = torchmetrics.AUROC(task="multiclass", num_classes=NUM_CLASSES)

    def forward(self, data: torch.Tensor) -> torch.Tensor:
        return self.model(data)

    def _common_step(self, batch: List[torch.Tensor], batch_idx: int) -> List[torch.Tensor]:
        feature, target = batch
        output = self.forward(feature)
        loss = self.loss_fn(output, target)
        return loss, output, target

    def training_step(self, batch: List[torch.Tensor], batch_idx: int) -> dict:
        loss, output, target = self._common_step(batch, batch_idx)
        self.log_dict(
            {
                "train_loss": loss,
            },
            on_step=True,
            on_epoch=True,
            prog_bar=True,
        )
        # return {"loss": loss, "output": output, "target": target}

    # TODO: THE CORRECT VERSION
    def validation_step(self, batch: List[torch.Tensor], batch_idx: int) -> dict:
        loss, output, target = self._common_step(batch, batch_idx)
        self.log_dict(
            {
                "val_loss": loss,
                "val_acc": self.accuracy(output, target),
                "val_f1": self.f1_score(output, target),
                "val_recall": self.recall(output, target),
                "val_precision": self.precision(output, target),
                # "val_auc": self.auc(outputs, targets),
            },
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )

        return {"loss": loss, "output": output, "target": target}

    # def validation_step(self, batch: List[torch.Tensor], batch_idx: int) -> dict:
    #     loss, output, target = self._common_step(batch, batch_idx)
    #     import random
    #     y_pred = torch.Tensor([random.randint(0, 2) for _ in range(50)])
    #     y_true = torch.Tensor([random.randint(0, 2) for _ in range(50)])
    #     # self.validation_step_outputs.append(
    #     #     {
    #     #         "loss": loss, 
    #     #         "output": output, 
    #     #         "target": target
    #     #     }   
    #     # )

    #     self.log_dict(
    #         {
    #             "val_loss": loss,
    #             "val_acc": self.accuracy(y_pred, y_true),
    #             "val_f1": self.f1_score(y_pred, y_true),
    #             "val_recall": self.recall(y_pred, y_true),
    #             "val_precision": self.precision(y_pred, y_true),
    #             # "val_auc": self.auc(outputs, targets),
    #         },
    #         on_step=False,
    #         on_epoch=True,
    #         prog_bar=True,
    #     )

    
    def test_step(self, batch: List[torch.Tensor], batch_idx: int) -> torch.Tensor:
        loss, output, target = self._common_step(batch, batch_idx)
        self.log("test_loss", loss)
        return loss

    def predict_step(self, batch: List[torch.Tensor], batch_idx: int) -> torch.Tensor:
        feature, _ = batch
        feature = feature.reshape(feature.size(0), -1)
        output = self.forward(feature)
        # TODO: check if categorical or sparse labels are needed
        preds = torch.argmax(output, dim=-1)
        return preds
    
    # # def on_train_epoch_end(self, outputs: torch.Tensor) -> None:
    # def on_train_epoch_end(self, outputs: List[dict]) -> None:
    #     outputs = torch.cat([x["output"] for x in outputs])
    #     targets = torch.cat([x["target"] for x in outputs])
    #     self.log_dict(
    #         {
    #             "train_acc": self.accuracy(outputs, targets),
    #             "train_f1": self.f1_score(outputs, targets),
    #             "train_recall": self.recall(outputs, targets),
    #             "train_precision": self.precision(outputs, targets),
    #             "train_auc": self.auc(outputs, targets),
    #         },
    #         on_step=False,
    #         on_epoch=True,
    #         prog_bar=True,
    #     )

    # def on_validation_epoch_end(self) -> None:
    # # def on_validation_epoch_end(self, outputs: torch.Tensor) -> None:
    #     outputs = torch.stack([x['output'] for x in self.validation_step_outputs])
    #     targets = torch.stack([x['target'] for x in self.validation_step_outputs])
    #     losses = torch.stack([x['loss'] for x in self.validation_step_outputs])

    #     self.log_dict(
    #         {
    #             "val_loss": losses.mean(),
    #             "val_acc": self.accuracy(outputs, targets),
    #             "val_f1": self.f1_score(outputs, targets),
    #             "val_recall": self.recall(outputs, targets),
    #             "val_precision": self.precision(outputs, targets),
    #             # "val_auc": self.auc(outputs, targets),
    #         },
    #         on_step=False,
    #         on_epoch=True,
    #         prog_bar=True,
    #     )
    
    def configure_optimizers(self) -> optim.Optimizer:
        return optim.Adam(self.parameters(), lr=self.learning_rate)


In [42]:
# import random
# y_pred = torch.Tensor([random.randint(0, 2) for _ in range(50)])
# y_true = torch.Tensor([random.randint(0, 2) for _ in range(50)])
# accuracy = torchmetrics.Recall(task="multiclass", num_classes=3)

# acc = accuracy(y_pred, y_true)
# acc

tensor(0.4200)

In [52]:
base_model_name = 'efficientnet_b0'
model = LightningNetwork(
        base_model=get_base_model(base_model_name),
        dropout=0.2, 
        output_dims=[128, 64],
        learning_rate=0.002
)
data_module = DataModule(model_name=base_model_name)

# logger = TensorBoardLogger(
#     save_dir="../logs/tb_logs", 
#     name="trail_v1"
# )
# profiler = PyTorchProfiler(
#     on_trace_ready=torch.profiler.tensorboard_trace_handler("../logs/tb_logs/profiler0"),
#     schedule=torch.profiler.schedule(skip_first=10, wait=1, warmup=1, active=20),
# )


trainer = pl.Trainer(
    # profiler=profiler,
    # logger=logger,
    # accelerator=ACCELERATOR,
    # devices=DEVICES,
    min_epochs=1,
    # max_steps=2,
    max_epochs=NUM_EPOCHS,
    precision='bf16-mixed',
    # precision=32,
    fast_dev_run=True
    # callbacks=[EarlyStopping(monitor="val_loss")],
)
trainer.fit(model, data_module)
trainer.validate(model, data_module)
# trainer.test(model, data_module)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.

  | Name      | Type                | Params
--------------------------------------------------
0 | model     | Network             | 4.2 M 
1 | loss_fn   | CrossEntropyLoss    | 0     
2 | accuracy  | MulticlassAccuracy  | 0     
3 | recall    | MulticlassRecall    | 0     
4 | precision | MulticlassPrecision | 0     
5 | f1_score  | MulticlassF1Score   | 0     
6 | auc       | MulticlassAUROC     | 0     
--------------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.720    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc                    0.0
         val_f1                     0.0
        val_loss             1.089205026626587
      val_precision                 0.0
       val_recall                   0.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 1.089205026626587,
  'val_acc': 0.0,
  'val_f1': 0.0,
  'val_recall': 0.0,
  'val_precision': 0.0}]

In [ ]:
import torch
import pytorch_lightning as pl
from model import NN
from dataset import MnistDataModule
import config
#TODO: import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.profilers import PyTorchProfiler

torch.set_float32_matmul_precision("medium") # to make lightning happy

if __name__ == "__main__":
    logger = TensorBoardLogger("tb_logs", name="mnist_model_v1")
    profiler = PyTorchProfiler(
        on_trace_ready=torch.profiler.tensorboard_trace_handler("tb_logs/profiler0"),
        schedule=torch.profiler.schedule(skip_first=10, wait=1, warmup=1, active=20),
    )
    model = NN(
        input_size=config.INPUT_SIZE,
        learning_rate=config.LEARNING_RATE,
        num_classes=config.NUM_CLASSES,
    )
    dm = MnistDataModule(
        data_dir=config.DATA_DIR,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )
    trainer = pl.Trainer(
        profiler=profiler,
        logger=logger,
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=1,
        max_epochs=config.NUM_EPOCHS,
        precision=config.PRECISION,
        # callbacks=[EarlyStopping(monitor="val_loss")],
    )
    trainer.fit(model, dm)
    trainer.validate(model, dm)
    trainer.test(model, dm)

In [2]:
img_paths = glob(f'{DATA_DIR}/images/*')
df = pd.read_csv(f'{DATA_DIR}/verified_annotation_from_xml.csv')

df = pd.read_csv(f'{DATA_DIR}/verified_annotation_from_xml.csv')
df['img_path'] =f'{DATA_DIR}/images/' + df['image_name']
df.drop(columns=['Unnamed: 0'], inplace=True)
df['label_name'] = df['label_name'].apply(lambda x: x.lower())
df['sparse_label'] = df['label_name'].map({'atopic': 0, 'papular': 1,'scabies': 2})
df.head()


,label_name,xmin,ymin,xmax,ymax,image_name,image_width,image_height,patient_id,img_path,sparse_label
0,papular,0,919,1635,2918,00241a_A_P.jpg,1960,4032,3.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,1
1,scabies,1006,1260,1713,2143,00240b_A_S.jpg,1960,4032,6.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,2
2,atopic,215,424,457,706,0052b_A_A.jpg,720,1280,9.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,0
3,scabies,138,94,633,643,0044a_A_S.jpg,1280,1280,14.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,2
4,scabies,354,866,950,1217,0044a_A_S.jpg,1280,1280,14.0,D:/Projects/Skin Disease Detection/Dataset/Lat...,2


In [4]:
import torch
from torch import nn
from typing import List
import torch.nn.functional as F
from torch import nn, optim
import pytorch_lightning as pl
import torchmetrics

import os
import sys
sys.path.append(os.pardir)
from src.config import *


In [ ]:
LightningNetwork(
    base_model = efficientnet_b0(),
    dropout=0.3, 
    output_dims=[128, 64])

In [ ]:
import torch
import pytorch_lightning as pl
from model import NN
from dataset import MnistDataModule
import config
#TODO: import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.profilers import PyTorchProfiler

torch.set_float32_matmul_precision("medium") # to make lightning happy

if __name__ == "__main__":
    logger = TensorBoardLogger("tb_logs", name="mnist_model_v1")
    profiler = PyTorchProfiler(
        on_trace_ready=torch.profiler.tensorboard_trace_handler("tb_logs/profiler0"),
        schedule=torch.profiler.schedule(skip_first=10, wait=1, warmup=1, active=20),
    )
    model = NN(
        input_size=config.INPUT_SIZE,
        learning_rate=config.LEARNING_RATE,
        num_classes=config.NUM_CLASSES,
    )
    dm = MnistDataModule(
        data_dir=config.DATA_DIR,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )
    trainer = pl.Trainer(
        profiler=profiler,
        logger=logger,
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=1,
        max_epochs=config.NUM_EPOCHS,
        precision=config.PRECISION,
        # callbacks=[EarlyStopping(monitor="val_loss")],
    )
    trainer.fit(model, dm)
    trainer.validate(model, dm)
    trainer.test(model, dm)

In [ ]:
import torch
import pytorch_lightning as pl
from model import NN
from dataset import MnistDataModule
import config
#TODO: import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.profilers import PyTorchProfiler

torch.set_float32_matmul_precision("medium") # to make lightning happy

if __name__ == "__main__":
    logger = TensorBoardLogger(
        save_dir="../logs/tb_logs", 
        name="trail_v1"
    )
    profiler = PyTorchProfiler(
        on_trace_ready=torch.profiler.tensorboard_trace_handler("../logs/tb_logs/profiler0"),
        schedule=torch.profiler.schedule(skip_first=10, wait=1, warmup=1, active=20),
    )
    base_model_name = 'efficientnet_b0'
    model = LightningNetwork(
            base_model=base_model_dict[base_model_name] ,
            dropout=0.2, 
            output_dims=[128, 64]
)
    dm = MnistDataModule(
        data_dir=config.DATA_DIR,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )
    trainer = pl.Trainer(
        profiler=profiler,
        logger=logger,
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=1,
        max_epochs=config.NUM_EPOCHS,
        precision=config.PRECISION,
        # callbacks=[EarlyStopping(monitor="val_loss")],
    )
    trainer.fit(model, dm)
    trainer.validate(model, dm)
    trainer.test(model, dm)


    
if __name__ == "__main__":
    logger = TensorBoardLogger("tb_logs", name="mnist_model_v1")
    profiler = PyTorchProfiler(
        on_trace_ready=torch.profiler.tensorboard_trace_handler("tb_logs/profiler0"),
        schedule=torch.profiler.schedule(skip_first=10, wait=1, warmup=1, active=20),
    )
    base_model_name = 'efficientnet_b0'
    model = LightningNetwork(
            base_model=base_model_dict[base_model_name] ,
            dropout=0.2, 
            output_dims=[128, 64]
)
    dm = MnistDataModule(
        data_dir=config.DATA_DIR,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )
    trainer = pl.Trainer(
        profiler=profiler,
        logger=logger,
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=1,
        max_epochs=config.NUM_EPOCHS,
        precision=config.PRECISION,
        # callbacks=[EarlyStopping(monitor="val_loss")],
    )
    trainer.fit(model, dm)
    trainer.validate(model, dm)
    trainer.test(model, dm)

